In [3]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import io
from googleapiclient.http import MediaIoBaseDownload
from tqdm import tqdm
from IPython.display import clear_output

In [4]:
# OAuth 2.0 클라이언트 비밀 파일 경로
SCOPES = ['https://www.googleapis.com/auth/drive']

# 구글 드라이브 API 인증을 위한 클라이언트 비밀 파일 경로를 찾는 함수
def find_credentials_file(api_folder_path):
    for file_name in os.listdir(api_folder_path):
        if file_name.endswith('.json'):
            return os.path.join(api_folder_path, file_name)
    raise FileNotFoundError("API 인증 JSON 파일을 찾을 수 없습니다.")

# 구글 드라이브 API 서비스 객체 생성
def create_service(api_folder_path):
    credentials_file = find_credentials_file(api_folder_path)
    
    # OAuth 2.0 클라이언트 인증
    flow = InstalledAppFlow.from_client_secrets_file(
        credentials_file, SCOPES)
    creds = flow.run_local_server(port=0)
    
    service = build('drive', 'v3', credentials=creds)
    return service


In [5]:
# API 폴더 경로 설정
api_folder_path = 'API'  # API 폴더 경로 설정
service = create_service(api_folder_path)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1073184021725-j15k2rnf2pa1lddcn2gr4pkpjg1eb8gq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8860%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=4ur1XXJ6FQoijtdWpJoLPDDRWNJXEz&access_type=offline


In [6]:
# 공유 폴더 ID를 추출하는 함수
def extract_folder_id_from_url(url):
    return url.split('/')[-1]

In [7]:
shared_folder_url = 'https://drive.google.com/drive/folders/1X3OEJXF5gzvjR5vRG7VO7h58bRwYTV0a'
shared_folder_id = extract_folder_id_from_url(shared_folder_url)

In [12]:
def clear_console():
    # Windows
    if os.name == 'nt':
        os.system('cls')
        clear_output(wait=True)
    # Unix 계열(Linux, macOS)
    else:
        os.system('clear')
        clear_output(wait=True)

def get_file_size(service, file_id):
    # Google Drive에서 파일의 크기 정보를 가져옵니다.
    file_info = service.files().get(fileId=file_id, fields="size", supportsAllDrives=True).execute()
    file_size = file_info.get('size')
    if file_size is None:
        print(f"File ID {file_id} has no size (possibly Google Docs or other unsupported format).")
        return 0  # 크기가 없으면 0을 반환
    
    return int(file_size)

# 파일을 다운로드하는 함수
def download_file(service, file_id, file_name, output_folder):
    file_path = os.path.join(output_folder, file_name)

    # Google Drive에서 파일 크기를 가져옵니다.
    drive_file_size = get_file_size(service, file_id)

    # 로컬에 파일이 이미 존재하는지 확인
    if os.path.exists(file_path):
        local_file_size = os.path.getsize(file_path)

        # 로컬 파일 크기와 Google Drive 파일 크기를 비교
        if local_file_size == drive_file_size:
            print(f"Skipping {file_name}, already exists and size matches.")
            return None
        else:
            print(f"{file_name} exists but size differs, downloading again.")
    
    # 파일 다운로드
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(file_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        pbar = tqdm(total=100, desc=f"Downloading {file_name}", unit='%')  # tqdm 초기화
        while not done:
            status, done = downloader.next_chunk()
            pbar.update(int(status.progress() * 100) - pbar.n)  # 업데이트를 통해 진행률 표시
        pbar.close()
    
    print(f"{file_name} has been downloaded to {file_path}.")
    return file_path  # 다운로드된 파일 경로 반환

# 폴더 내용을 재귀적으로 다운로드하는 함수
def download_folder(service, folder_id, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    results = service.files().list(
        q=f"'{folder_id}' in parents",
        fields="files(id, name, mimeType)",
        supportsAllDrives=True,
        includeItemsFromAllDrives=True
    ).execute()
    items = results.get('files', [])

    downloaded_files = []  # 다운로드된 파일 경로를 저장할 리스트

    for item in items:
        file_id = item['id']
        file_name = item['name']
        mime_type = item['mimeType']
        
        if mime_type == 'application/vnd.google-apps.folder':
            print(f"Entering folder: {file_name}")
            new_output_folder = os.path.join(output_folder, file_name)
            # 하위 폴더의 파일들도 다운로드하고, 리스트에 추가
            downloaded_files.extend(download_folder(service, file_id, new_output_folder))
        else:
            print(f"Downloading file: {file_name}")
            downloaded_file_path = download_file(service, file_id, file_name, output_folder)
            if downloaded_file_path:
                downloaded_files.append(downloaded_file_path)
        
        clear_console()
    
    return downloaded_files  # 다운로드된 파일들의 경로 반환


In [13]:
output_folder = 'data'
downloaded_files = download_folder(service, shared_folder_id, output_folder)

Entering folder: P031_split2
Skipping P031_split2.b3d, already exists and size matches.


In [15]:
downloaded_files

['data\\test\\With_Arm\\vanderZee2022_Formatted_With_Arm\\p3\\p3.b3d',
 'data\\train\\No_Arm\\Camargo2021_Formatted_No_Arm\\AB17_split4\\AB17_split4.b3d',
 'data\\train\\No_Arm\\Santos2017_Formatted_No_Arm\\subject17\\subject17.b3d']